# Parallel and Distributed Low Depth Algorithms for QAE

This project implements a parallelized and distributed version of the Power Law algorithm for quantum amplitude estimation. The Power Law algorithm is a low depth algorithm utilizing maximum likelihood estimation based on the measurements produced from different quantum circuits.

Power Law paper: https://arxiv.org/abs/2012.03348